<a href="https://colab.research.google.com/github/BlackWyvernX/GPCR-Ligand-Dock/blob/main/Glass_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q selenium

In [ ]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests as rq
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from Glass_Scrape import scrape_ligand_table

In [ ]:
!apt-get update
!apt-get install -y chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

# Set options for Chrome WebDriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# Initialize the WebDriver with the specified options
driver = webdriver.Chrome(options=chrome_options)

In [ ]:
df_pandas = pd.read_html(url, attrs = {'class': "g_table g_query_table"},  flavor='bs4', thousands ='.')

In [ ]:
df = pd.DataFrame(df_pandas[0])

In [ ]:
df

In [ ]:
df.loc[df['Receptor'] == '5-hydroxytryptamine receptor 1A']

In [ ]:
table_rows = tables.find_all('tr')

receptor_links = []

for row in table_rows[1:]:
    # Extract the receptor name and its link
    receptor_data = row.find_all('td')
    receptor_name = receptor_data[1].find('a').text
    receptor_link = receptor_data[1].find('a')['href']

    # Append the receptor name and link to the list
    receptor_links.append((receptor_name, receptor_link))

# Print the list of receptor names and links
for name, link in receptor_links:
    print(f"Receptor Name: {name}")
    print(f"Receptor Link: {link}")
    print("\n")

In [ ]:
receptor_name = "Neuropeptide S receptor"

receptor_link = None
for name, link in receptor_links:
    if name == receptor_name:
        receptor_link = link
        break

In [ ]:
if receptor_link:
    receptor_url = f'https://zhanggroup.org/GLASS/{receptor_link}'

    receptor_data = rq.get(receptor_url).text
    receptor_soup = bs(receptor_data, 'html.parser')

    ligand_table = receptor_soup.find('table', class_='g_table g_query_table')

    if ligand_table:
        ligand_data = pd.read_html(str(ligand_table))

        ligand_df = pd.DataFrame(ligand_data[0])
    else:
        print(f"Ligand Table not found for {receptor_name}")
else:
    print(f"Receptor '{receptor_name}' not found in the list")

In [ ]:
def scrape_ligand_table(target_receptor_name):
    url = 'https://zhanggroup.org/GLASS/query.cgi?dataset=Search+by+GLASS+ID'
    data = rq.get(url).text
    soup = bs(data, 'html.parser')

    tables = soup.find('table', {'class': "g_table g_query_table"})

    table_rows = tables.find_all('tr')

    receptor_links = []

    for row in table_rows[1:]:
        receptor_data = row.find_all('td')
        receptor_name = receptor_data[1].find('a').text
        receptor_link = receptor_data[1].find('a')['href']

        receptor_links.append((receptor_name, receptor_link))

    receptor_link = None
    for name, link in receptor_links:
        if name == target_receptor_name:
            receptor_link = link
            break

    if receptor_link:
        receptor_url = f'https://zhanggroup.org/GLASS/{receptor_link}'

        receptor_data = rq.get(receptor_url).text
        receptor_soup = bs(receptor_data, 'html.parser')

        ligand_table = receptor_soup.find('table', class_='g_table g_query_table')

        if ligand_table:
            ligand_data = pd.read_html(str(ligand_table))
            ligand_df = pd.DataFrame(ligand_data[0])
            return ligand_df
        else:
            print(f"Ligand Table not found for {target_receptor_name}")
    else:
        print(f"Receptor '{target_receptor_name}' not found in the list")


In [ ]:
top = scrape_ligand_table('5-hydroxytryptamine receptor 1A')

In [ ]:
top

,GLASS ID,Molecule,Formula,Molecular weight,H-bond acceptor / donor,XlogP,Lipinski's druglikeness
0,37,MLS000077136,C12H11N3O6,293.235,8 / 1,-0.1,Yes
1,557309,"3-(5-(4-(4-(3,5-difluoro-2-oxopyridin-1(2H)-yl...",C29H29F2N5O,501.582,6 / 1,5.1,No
2,91,Eticlopride,C17H25ClN2O3,340.848,4 / 2,3.1,Yes
3,98,AC1O8PYY,C17H25ClN2O3,340.848,4 / 2,3.1,Yes
4,172,CHEMBL2111779,C24H25N3O2,387.483,4 / 1,3.8,Yes
...,...,...,...,...,...,...,...
95,7345,CHEMBL179648,C21H26N2O,322.452,3 / 0,3.7,Yes
96,7450,CHEMBL377697,C25H40N4O3S,476.680,6 / 2,3.7,Yes
97,442020,CHEMBL3342852,C19H28N2O3S,364.504,6 / 0,2.5,Yes
98,547980,CHEMBL3983496,C32H41N3O4,531.697,6 / 1,5.2,No


In [ ]:
df_fil = df[df["Lipinski's druglikeness"] == 'Yes']

In [ ]:
df_fil.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21 entries, 2 to 48
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   GLASS ID                 21 non-null     int64  
 1   Molecule                 21 non-null     object 
 2   Formula                  21 non-null     object 
 3   Molecular weight         21 non-null     int64  
 4   H-bond acceptor / donor  21 non-null     object 
 5   XlogP                    21 non-null     float64
 6   Lipinski's druglikeness  21 non-null     object 
dtypes: float64(1), int64(2), object(4)
memory usage: 1.3+ KB


In [ ]:
df_lig = df_fil['Molecule']

In [ ]:
df_lig.to_csv('GPR17Ligand.csv')